In [1]:
from Decoder_RNN import DecoderRNN
from Encoder_CNN import EncoderCNN
import torch.nn as nn
import torch
import torch.optim as optim
import kagglehub
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from PIL import Image
import os
import spacy
import torchvision.transforms.v2 as transforms
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm

/home/esun/Image_Captioner/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Download latest version
#path = kagglehub.dataset_download("adityajn105/flickr8k")

#print("Path to dataset files:", path)
#/home/esun/.cache/kagglehub/datasets/adityajn105/flickr8k/versions/1

In [3]:
class CNNtoRNN(nn.Module):
    def __init__(self, embed_size, hidden_size, vocab_size, num_layers):
        super(CNNtoRNN, self).__init__()
        self.encoderCNN = EncoderCNN(embed_size)
        self.decoderRNN = DecoderRNN(embed_size, hidden_size, vocab_size, num_layers)

    def forward(self, images, captions):
        features = self.encoderCNN(images)
        outputs = self.decoderRNN(features, captions)
        return outputs

    def caption_image(self, image, vocabulary, max_length=50):
        result_caption = []

        with torch.no_grad():
            x = self.encoderCNN(image).unsqueeze(0)
            states = None

            for _ in range(max_length):
                hiddens, states = self.decoderRNN.lstm(x, states)
                output = self.decoderRNN.linear(hiddens.squeeze(0))
                predicted = output.argmax(1)
                result_caption.append(predicted.item())
                x = self.decoderRNN.embed(predicted).unsqueeze(0)

                if vocabulary.itos[predicted.item()] == "<EOS>":
                    break

        return [vocabulary.itos[idx] for idx in result_caption]

In [4]:
class Vocabulary:
    def __init__(self, freq_threshold):
        self.itos = {0: "<PAD>", 1: "<SOS>", 2: "<EOS>", 3: "<UNK>"}
        self.stoi = {"<PAD>": 0, "<SOS>": 1, "<EOS>": 2, "<UNK>": 3}
        self.freq_threshold = freq_threshold

    def __len__(self):
        return len(self.itos)

    @staticmethod
    def tokenizer_eng(text):
        return [tok.text.lower() for tok in spacy_eng.tokenizer(text)]

    def build_vocabulary(self, sentence_list):
        frequencies = {}
        idx = 4

        for sentence in sentence_list:
            for word in self.tokenizer_eng(sentence):
                if word not in frequencies:
                    frequencies[word] = 1

                else:
                    frequencies[word] += 1

                if frequencies[word] == self.freq_threshold:
                    self.stoi[word] = idx
                    self.itos[idx] = word
                    idx += 1

    def numericalize(self, text):
        tokenized_text = self.tokenizer_eng(text)

        return [
            self.stoi[token] if token in self.stoi else self.stoi["<UNK>"]
            for token in tokenized_text
        ]

In [5]:
class FlickrDataset(Dataset):
    def __init__(self, root_dir, captions_file, transform=None, freq_threshold=5):
        self.root_dir = root_dir
        self.df = pd.read_csv(captions_file)
        self.transform = transform

        # Get img, caption columns
        self.imgs = self.df["image"]
        self.captions = self.df["caption"]

        # Initialize vocabulary and build vocab
        self.vocab = Vocabulary(freq_threshold)
        self.vocab.build_vocabulary(self.captions.tolist())

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        caption = self.captions[index]
        img_id = self.imgs[index]
        img = Image.open(os.path.join(self.root_dir, img_id)).convert("RGB")

        if self.transform is not None:
            img = self.transform(img)

        numericalized_caption = [self.vocab.stoi["<SOS>"]]
        numericalized_caption += self.vocab.numericalize(caption)
        numericalized_caption.append(self.vocab.stoi["<EOS>"])

        return img, torch.tensor(numericalized_caption)

In [6]:
spacy_eng = spacy.load("en_core_web_sm")

In [7]:
class MyCollate:
    def __init__(self, pad_idx):
        self.pad_idx = pad_idx

    def __call__(self, batch):
        imgs = [item[0].unsqueeze(0) for item in batch]
        imgs = torch.cat(imgs, dim=0)
        targets = [item[1] for item in batch]
        targets = pad_sequence(targets, batch_first=False, padding_value=self.pad_idx)

        return imgs, targets

In [8]:
root="Flickr8k/Images/"
captions="Flickr8k/captions.txt"
transform = transforms.Compose(
        [
            transforms.Resize((224, 224)),
            transforms.RandomResizedCrop((224, 224)),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ]
    )
dataset=FlickrDataset(root,captions,transform)
pad_idx=dataset.vocab.stoi["<PAD>"]
loader=DataLoader(dataset,batch_size=32,num_workers=2,shuffle=True,pin_memory=True,collate_fn=MyCollate(pad_idx))

/home/esun/Image_Captioner/.venv/lib/python3.12/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [9]:
for idx,(imgs,captions) in enumerate(loader):
    print(imgs)
    print(captions)
    break

tensor([[[[ 0.2314,  0.0118, -0.4667,  ..., -0.0667,  0.1922,  0.6392],
          [ 0.3412,  0.0902, -0.4196,  ...,  0.4275,  0.6314,  0.7176],
          [ 0.1137,  0.1294, -0.2000,  ...,  0.3961,  0.6627,  0.6078],
          ...,
          [ 0.4980,  0.5137,  0.5529,  ..., -0.8275, -0.8039, -0.8039],
          [ 0.3882,  0.3647,  0.3882,  ..., -0.8745, -0.7882, -0.8039],
          [ 0.2078,  0.1608,  0.1686,  ..., -0.7333, -0.6706, -0.6784]],

         [[ 0.0902, -0.0431, -0.4588,  ...,  0.2471,  0.4353,  0.7804],
          [ 0.2078,  0.0431, -0.4196,  ...,  0.5922,  0.7647,  0.8353],
          [ 0.0118,  0.0902, -0.2078,  ...,  0.5922,  0.8039,  0.7725],
          ...,
          [ 0.5137,  0.5216,  0.5451,  ..., -0.8510, -0.8196, -0.8039],
          [ 0.4353,  0.4039,  0.4118,  ..., -0.8824, -0.8039, -0.8118],
          [ 0.2784,  0.2157,  0.2235,  ..., -0.7647, -0.7098, -0.6941]],

         [[-0.0275, -0.0902, -0.5608,  ..., -0.4745, -0.2863, -0.0039],
          [ 0.1216, -0.0039, -

In [10]:
def print_examples(model, device, dataset):
    transform = transforms.Compose(
        [
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ]
    )

    model.eval()
    test_img1 = transform(Image.open("test_examples/dog.jpg").convert("RGB")).unsqueeze(
        0
    )
    print("Example 1 CORRECT: Dog on a beach by the ocean")
    print(
        "Example 1 OUTPUT: "
        + " ".join(model.caption_image(test_img1.to(device), dataset.vocab))
    )
    test_img2 = transform(
        Image.open("test_examples/child.jpg").convert("RGB")
    ).unsqueeze(0)
    print("Example 2 CORRECT: Child holding red frisbee outdoors")
    print(
        "Example 2 OUTPUT: "
        + " ".join(model.caption_image(test_img2.to(device), dataset.vocab))
    )
    test_img3 = transform(Image.open("test_examples/bus.png").convert("RGB")).unsqueeze(
        0
    )
    print("Example 3 CORRECT: Bus driving by parked cars")
    print(
        "Example 3 OUTPUT: "
        + " ".join(model.caption_image(test_img3.to(device), dataset.vocab))
    )
    test_img4 = transform(
        Image.open("test_examples/boat.png").convert("RGB")
    ).unsqueeze(0)
    print("Example 4 CORRECT: A small boat in the ocean")
    print(
        "Example 4 OUTPUT: "
        + " ".join(model.caption_image(test_img4.to(device), dataset.vocab))
    )
    test_img5 = transform(
        Image.open("test_examples/horse.png").convert("RGB")
    ).unsqueeze(0)
    print("Example 5 CORRECT: A cowboy riding a horse in the desert")
    print(
        "Example 5 OUTPUT: "
        + " ".join(model.caption_image(test_img5.to(device), dataset.vocab))
    )
    model.train()

In [ ]:
torch.backends.cudnn.benchmark = True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
load_model = False
save_model = False
train_CNN = False
# Hyperparameters
embed_size = 256
hidden_size = 256
vocab_size = len(dataset.vocab)
num_layers = 1
learning_rate = 3e-4
num_epochs = 128
step = 0



# initialize model, loss etc
model = CNNtoRNN(embed_size, hidden_size, vocab_size, num_layers).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=dataset.vocab.stoi["<PAD>"])
optimizer = optim.Adam(model.parameters(), lr=learning_rate)




# Only finetune the CNN
for name, param in model.encoderCNN.resnet50.named_parameters():
    if "fc.weight" in name or "fc.bias" in name:
        param.requires_grad = True
    else:
        param.requires_grad = train_CNN



model.train()


for epoch in range(num_epochs):
    print(epoch)
    # Uncomment the line below to see a couple of test cases
    print_examples(model, device, dataset)
    
    for idx, (imgs, captions) in tqdm(
        enumerate(loader), total=len(loader), leave=False
    ):
        imgs = imgs.to(device)
        captions = captions.to(device)
        outputs = model(imgs, captions[:-1])
        loss = criterion(
            outputs.reshape(-1, outputs.shape[2]), captions.reshape(-1)
        )
        step += 1
        optimizer.zero_grad()
        loss.backward(loss)
        optimizer.step()

0
Example 1 CORRECT: Dog on a beach by the ocean


/home/esun/Image_Captioner/.venv/lib/python3.12/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


Example 1 OUTPUT: ! crystal engine children spotlight jogs connected lab feathered interviews viewer swing ball cycling bends vehicles topless infant incline throwing crashes crashes vehicles ollie focus tag onstage beyond squats purse purse paper ascending low batting ascending community sand acrobatic rail listening swan carry filling mouse resort trunk guiding dancing carving
Example 2 CORRECT: Child holding red frisbee outdoors
Example 2 OUTPUT: males way bite very brownish gathers stairway ! overhead overhead overhead overhead overhead overhead overhead overhead overhead overhead overhead overhead overhead overhead overhead overhead overhead overhead overhead overhead overhead overhead overhead overhead overhead overhead overhead overhead overhead overhead overhead overhead overhead overhead overhead overhead overhead overhead overhead overhead overhead overhead
Example 3 CORRECT: Bus driving by parked cars
Example 3 OUTPUT: pizza winds tattoos climbing carpeted site being goose w

1
Example 1 CORRECT: Dog on a beach by the ocean
Example 1 OUTPUT: a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a
Example 2 CORRECT: Child holding red frisbee outdoors
Example 2 OUTPUT: a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a
Example 3 CORRECT: Bus driving by parked cars
Example 3 OUTPUT: a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a
Example 4 CORRECT: A small boat in the ocean
Example 4 OUTPUT: a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a
Example 5 CORRECT: A cowboy riding a horse in the desert
Example 5 OUTPUT: a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a


2
Example 1 CORRECT: Dog on a beach by the ocean
Example 1 OUTPUT: a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a
Example 2 CORRECT: Child holding red frisbee outdoors
Example 2 OUTPUT: a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a
Example 3 CORRECT: Bus driving by parked cars
Example 3 OUTPUT: a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a
Example 4 CORRECT: A small boat in the ocean
Example 4 OUTPUT: a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a
Example 5 CORRECT: A cowboy riding a horse in the desert
Example 5 OUTPUT: a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a a


...

30
Example 1 CORRECT: Dog on a beach by the ocean
Example 1 OUTPUT: <SOS> a man in a man in a <UNK> . <EOS>
Example 2 CORRECT: Child holding red frisbee outdoors
Example 2 OUTPUT: <SOS> a man in a man in a <UNK> . <EOS>
Example 3 CORRECT: Bus driving by parked cars
Example 3 OUTPUT: <SOS> a man in a man in a <UNK> . <EOS>
Example 4 CORRECT: A small boat in the ocean
Example 4 OUTPUT: <SOS> a man in a man in a <UNK> . <EOS>
Example 5 CORRECT: A cowboy riding a horse in the desert
Example 5 OUTPUT: <SOS> a man in a man in a <UNK> . <EOS>


31
Example 1 CORRECT: Dog on a beach by the ocean
Example 1 OUTPUT: <SOS> a man in a man in a <UNK> . <EOS>
Example 2 CORRECT: Child holding red frisbee outdoors
Example 2 OUTPUT: <SOS> a man in a man in a <UNK> . <EOS>
Example 3 CORRECT: Bus driving by parked cars
Example 3 OUTPUT: <SOS> a man in a man in a <UNK> . <EOS>
Example 4 CORRECT: A small boat in the ocean
Example 4 OUTPUT: <SOS> a man in a man in a <UNK> . <EOS>
Example 5 CORRECT: A cowboy riding a horse in the desert
Example 5 OUTPUT: <SOS> a man in a man in a <UNK> . <EOS>


32
Example 1 CORRECT: Dog on a beach by the ocean
Example 1 OUTPUT: <SOS> a man in a man in a <UNK> . <EOS>
Example 2 CORRECT: Child holding red frisbee outdoors
Example 2 OUTPUT: <SOS> a man in a man in a <UNK> . <EOS>
Example 3 CORRECT: Bus driving by parked cars
Example 3 OUTPUT: <SOS> a man in a man in a <UNK> . <EOS>
Example 4 CORRECT: A small boat in the ocean
Example 4 OUTPUT: <SOS> a man in a man in a <UNK> . <EOS>
Example 5 CORRECT: A cowboy riding a horse in the desert
Example 5 OUTPUT: <SOS> a man in a man in a <UNK> . <EOS>


33
Example 1 CORRECT: Dog on a beach by the ocean
Example 1 OUTPUT: <SOS> a man in a man in a <UNK> . <EOS>
Example 2 CORRECT: Child holding red frisbee outdoors
Example 2 OUTPUT: <SOS> a man in a man in a <UNK> . <EOS>
Example 3 CORRECT: Bus driving by parked cars
Example 3 OUTPUT: <SOS> a man in a man in a <UNK> . <EOS>
Example 4 CORRECT: A small boat in the ocean
Example 4 OUTPUT: <SOS> a man in a man in a <UNK> . <EOS>
Example 5 CORRECT: A cowboy riding a horse in the desert
Example 5 OUTPUT: <SOS> a man in a man in a <UNK> . <EOS>


KeyboardInterrupt: 

In [ ]:
print_examples(model,device,dataset)

Example 1 CORRECT: Dog on a beach by the ocean
Example 1 OUTPUT: <SOS> a dog walking on a beach . <EOS>
Example 2 CORRECT: Child holding red frisbee outdoors
Example 2 OUTPUT: <SOS> a little girl in a yellow dress and blue pants is in front of a blue and yellow merry - go - round . <EOS>
Example 3 CORRECT: Bus driving by parked cars
Example 3 OUTPUT: <SOS> a man is walking next to a building with a red car shaped like it is looking away . <EOS>
Example 4 CORRECT: A small boat in the ocean
Example 4 OUTPUT: <SOS> a boat floats in the water . <EOS>
Example 5 CORRECT: A cowboy riding a horse in the desert
Example 5 OUTPUT: <SOS> a man and a woman are sitting on a hill overlooking a lake . <EOS>


In [ ]:
torch.save(model.state_dict(), 'model_weights.pth')